In [15]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import OneHotEncoder
import gensim.downloader as api
import re
import swifter

# Enable tqdm for pandas
tqdm.pandas()

In [ ]:
# Load GloVe model
glove_model = api.load("glove-twitter-200")  # 200-dimensional GloVe embeddings

In [ ]:
# Function to compute the average word vector for a tweet
def get_avg_embedding(tweet, model, vector_size=200):
    words = tweet.split()  # Tokenize by whitespace
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:  # If no words in the tweet are in the vocabulary, return a zero vector
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)

# Preprocessing function
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenization
    words = text.split()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

KeyboardInterrupt: 

In [ ]:
# Load data
folder_path = "train_tweets"
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".csv")] # Only use first file for testing purposes
df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

# Apply preprocessing
df['Tweet'] = df['Tweet'].swifter.apply(preprocess_text)
print(df.head)

Pandas Apply: 100%|██████████| 5056050/5056050 [13:21<00:00, 6308.31it/s]


<bound method NDFrame.head of              ID  MatchID  PeriodID  EventType      Timestamp  \
0           2_0        2         0          0  1403538600000   
1           2_0        2         0          0  1403538600000   
2           2_0        2         0          0  1403538600000   
3           2_0        2         0          0  1403538600000   
4           2_0        2         0          0  1403538600000   
...         ...      ...       ...        ...            ...   
5056045  17_129       17       129          1  1403805600000   
5056046  17_129       17       129          1  1403805600000   
5056047  17_129       17       129          1  1403805600000   
5056048  17_129       17       129          1  1403805600000   
5056049  17_129       17       129          1  1403805600000   

                                                     Tweet  
0        rt soccerdotcom esp beat au well give away spa...  
1        visit sitep official web site httptcoehzkslan ...  
2        rt soccer

In [45]:
# Feature creation
# Add the length of each tweet as a feature
df['TweetLength'] = df['Tweet'].apply(len)

# Add a simple tweet count feature
df['TweetCount'] = df.groupby(['MatchID', 'PeriodID', 'Timestamp'])['Timestamp'].transform('count')

# Add word count as a feature
df['WordCount'] = df['Tweet'].apply(lambda x: len(x.split()))

print(df)

             ID  MatchID  PeriodID  EventType      Timestamp  \
0           2_0        2         0          0  1403538600000   
1           2_0        2         0          0  1403538600000   
2           2_0        2         0          0  1403538600000   
3           2_0        2         0          0  1403538600000   
4           2_0        2         0          0  1403538600000   
...         ...      ...       ...        ...            ...   
5056045  17_129       17       129          1  1403805600000   
5056046  17_129       17       129          1  1403805600000   
5056047  17_129       17       129          1  1403805600000   
5056048  17_129       17       129          1  1403805600000   
5056049  17_129       17       129          1  1403805600000   

                                                     Tweet  TweetLength  \
0        rt soccerdotcom esp beat au well give away spa...          104   
1        visit sitep official web site httptcoehzkslan ...           95   
2     

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Compute TF-IDF weights for the corpus
vectorizer = TfidfVectorizer(max_features=10000)
vectorizer.fit(df['Tweet'])
tfidf_weights = dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))

# Weighted average embeddings
def get_weighted_avg_embedding(tweet, model, vector_size=200, weights=tfidf_weights):
    words = tweet.split()
    word_vectors = [model[word] * weights.get(word, 1) for word in words if word in model]
    if not word_vectors:
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)


In [ ]:
# Generate embeddings for each tweet
vector_size = 200  # GloVe embedding dimension
tweet_vectors = df['Tweet'].swifter.apply(lambda tweet: get_weighted_avg_embedding(tweet, model=glove_model, vector_size=200, weights=tfidf_weights)
tweet_vectors = np.array(list(tweet_vectors), dtype=np.float32)

#tweet_vectors = df['Tweet'].swifter.apply(get_tweet_embedding)

Pandas Apply: 100%|██████████| 5056050/5056050 [02:47<00:00, 30234.03it/s]


In [47]:
tweet_df = pd.DataFrame(tweet_vectors)

# Attach the vectors into the original dataframe
period_features = pd.concat([df, tweet_df], axis=1)

# Drop the columns that are not useful anymore
period_features = period_features.drop(columns=['Timestamp', 'Tweet'])

print("X_train_reshaped shape:", period_features.shape)
# Group the tweets into their corresponding periods. This way we generate an average embedding vector for each period
period_features = period_features.groupby(['MatchID', 'PeriodID', 'ID']).mean().reset_index()

# We drop the non-numerical features and keep the embeddings values for each period
X = period_features.drop(columns=['EventType', 'MatchID', 'PeriodID', 'ID']).values
# We extract the labels of our training samples
y = period_features['EventType'].values


# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Add a time step dimension to match the LSTM input shape
X_train_reshaped = X_train[:, None, :]  # Add a new axis for timesteps
X_test_reshaped = X_test[:, None, :]    # Add a new axis for timesteps
print(X_train_reshaped)



# Define the LSTM model
model = Sequential([
    tf.keras.layers.Input(shape=(1, X_train_reshaped.shape[2])),  # Input layer
    LSTM(128, return_sequences=False),             # LSTM layer with 128 units
    Dense(y_encoded.shape[1], activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train_reshaped, y_train,
                    epochs=10,
                    batch_size=64,
                    validation_split=0.2,
                    verbose=1)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test, verbose=1)

print(f"Test Accuracy: {test_accuracy:.4f}")

X_train_reshaped shape: (5056050, 207)
[[[ 5.73310472e+01  6.43199683e+01  9.18517542e+00 ...  7.86279216e-02
   -2.27761175e-02  1.68399289e-01]]

 [[ 5.90684768e+01  1.40212100e+02  8.54710015e+00 ...  5.86179160e-02
   -1.33317336e-01  6.45858273e-02]]

 [[ 5.98365350e+01  4.75320768e+01  1.00427691e+01 ...  1.12150736e-01
   -9.48686674e-02  1.54207066e-01]]

 ...

 [[ 4.98721346e+01  8.01915878e+01  7.34447950e+00 ... -3.41409110e-02
   -8.03756118e-02  1.86816081e-01]]

 [[ 6.71224122e+01  3.80018002e+01  9.77677768e+00 ...  2.54440494e-03
   -8.70075002e-02  1.91855818e-01]]

 [[ 8.40793651e+01  9.08730159e+00  1.36230159e+01 ...  7.06264079e-02
   -7.17813224e-02  1.38182014e-01]]]
Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5202 - loss: 0.7052 - val_accuracy: 0.5526 - val_loss: 0.6808
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5687 - loss: 0.6758 - val_accuracy: 0.5731 - val_loss: 0.6758
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/ste

In [21]:
###### For Kaggle submission

predictions = []
dummy_predictions = []
# We read each file separately, we preprocess the tweets and then use the classifier to predict the labels.
# Finally, we concatenate all predictions into a list that will eventually be concatenated and exported
# to be submitted on Kaggle.
for fname in os.listdir("eval_tweets"):
    val_df = pd.read_csv("eval_tweets/" + fname)
    
    val_df['Tweet'] = val_df['Tweet'].swifter.apply(preprocess_text)

    tweet_vectors = val_df['Tweet'].swifter.apply(lambda x: get_avg_embedding(x, glove_model, vector_size))

    tweet_vectors = np.array(list(tweet_vectors), dtype=np.float32)

    tweet_df = pd.DataFrame(tweet_vectors)

    period_features = pd.concat([val_df, tweet_df], axis=1)
    period_features = period_features.drop(columns=['Timestamp', 'Tweet'])
    period_features = period_features.groupby(['MatchID', 'PeriodID', 'ID']).mean().reset_index()

    X = period_features.drop(columns=['MatchID', 'PeriodID', 'ID']).values

    # Reshape input for LSTM
    X_reshaped = X[:, None, :]  # Add timestep dimension

    preds = model.predict(X_reshaped)
    preds = preds.argmax(axis=1)  # Convert probabilities to class indices
    period_features['EventType'] = preds
    predictions.append(period_features[['ID', 'EventType']])


pred_df = pd.concat(predictions)
pred_df.to_csv('LSTM_predictions.csv', index=False)



Pandas Apply: 100%|██████████| 285804/285804 [00:09<00:00, 30354.72it/s]


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


Pandas Apply: 100%|██████████| 45024/45024 [00:01<00:00, 32031.69it/s]


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


Pandas Apply:  33%|███▎      | 207754/628698 [00:34<01:09, 6099.18it/s]


KeyboardInterrupt: 